![](https://habrastorage.org/webt/ea/zz/g7/eazzg7inp28p1osmj25fc6e1mgo.jpeg)

![](https://habrastorage.org/webt/v-/ny/wv/v-nywv6nm3uipk2cfhygvgievi0.png)

![](https://habrastorage.org/webt/bo/ln/ie/bolnieqllmfkdenqydhyjcewace.jpeg)

![](https://habrastorage.org/webt/qu/yd/xh/quydxhdullq7bho7iicmrraf5rk.png)

In [ ]:
using Statistics: mean

In [ ]:
using Plots, CSV, DataFrames#, Interact

In [ ]:
apples = CSV.read("data/Apple_Braeburn.csv")
banans = CSV.read("data/bananas.csv")

In [ ]:
Desc = describe(apples, :all)

In [ ]:
Desc[5, :]

In [ ]:
function plot2features(clr)
    
    x_apples = apples[:, :green]
    x_banans = banans[:, :green]
    y_apples = apples[:, clr]
    y_banans = banans[:, clr]

    scatter(x_apples, y_apples, lab = "apples", colour = :red)
    scatter!(x_banans, y_banans, lab = "bananas", legend = :topleft, colour = :yellow)
    hline!([mean(y_apples), mean(y_banans) ], lab = "" )
    vline!([mean(x_apples), mean(x_banans) ], lab = "" )
    xaxis!("green")
    yaxis!("$clr")
end

plot2features(:red)

In [ ]:
plot2features(:blue)

In [ ]:
savefig("C:\\Users\\User\\Desktop\\blue_2d_plot")

In [ ]:
x_apples .-= minimum(x_apples)
x_banans .-= minimum(x_apples)
x_apples /= ( maximum(x_banans) - minimum(x_apples) )
x_banans /= ( maximum(x_banans) - minimum(x_apples) );

In [ ]:
plot2features(:blue)

In [ ]:
sigmo(x,w,b) = 1 / (1 + exp(-w*x+b))

r_apples, g_apples, b_apples = apples[:, :red], apples[:, :green], apples[:, :blue]
r_banans, g_banans, b_banans = banans[:, :red], banans[:, :green], banans[:, :blue];

In [ ]:
@manipulate for w in 10:1:60, b in -5:1:25
    
    plot(x->sigmo(x,w,b), 0, 1, label="Model", legend = :topleft, lw=3)
    scatter!(g_apples[1:5], zeros(10), label="Apple", colour = :red)
    scatter!(g_banans[1:5], ones(10), label="Banana", colour = :yellow)
    
end

In [ ]:
foon(x) = sigmo(x,60,24)
plot(foon, 0, 1, label="Model", legend = :topleft, lw=3)
scatter!(foon, g_apples, label="Apple", colour = :red)
scatter!(foon, g_banans, label="Banana", colour = :yellow)
xaxis!("green")

In [ ]:
savefig("C:/Users/User/Desktop/green.png")

In [ ]:
apples_mean_green = mean(g_apples)
banans_mean_green = mean(g_banans)

L(w, b) = (0 - sigmo(apples_mean_green,w,b))^2 + (1 - sigmo(banans_mean_green,w,b))^2

w_range = 10:0.5:30
b_range = 0:0.5:20

L_values = [L(w,b) for b in b_range, w in w_range]


@manipulate for w in w_range, b in b_range
    p1 = surface(w_range, b_range, L_values, xlabel="b", ylabel="w", cam=(80,40), cbar=false, leg=false)
    scatter!(p1, [w], [b], [L(w,b)+1e-2], markersize=5, color = :blue)
    p2 = plot(x->sigmo(x,w,b), 0, 1, label="Model", legend = :topleft, lw=3)
    scatter!(p2, [apples_mean_green],  [0.0], label="Apple", markersize=10)
    scatter!(p2, [banans_mean_green], [1.0], label="Banana", markersize=10, xlim=(0,1), ylim=(0,1))
    plot(p1, p2, layout=(2,1))
end

In [ ]:
σ(x,w,b) = 1 / (1 + exp(-(w*x)[1]+b))
y_apples = apples[:, :blue]
y_banans = banans[:, :blue]

@manipulate for w1 in 10:1:60, w2 in 10:1:60, b in 15:1:85
    
    σ2d = [σ([x1, x2], [w1 w2], b ) for x1 = 0.18:0.01:0.6, x2 = 0.18:0.01:0.6 ];

    contour([0.18:0.01:0.6;], [0.18:0.01:0.6;], σ2d, fill = true)
    scatter!(x_apples, y_apples, lab = "apples", colour = :red)
    scatter!(x_banans, y_banans, lab = "bananas", legend = :topleft, colour = :yellow)
end

# One feature

In [ ]:
using Flux: param, params, Dense, mse, train!, σ, onecold, throttle, Descent, Tracker.data

In [ ]:
Y = [zeros(length(g_apples)); ones(length(g_banans)) ] |> permutedims
X = [g_apples; g_banans] |> permutedims;

# dataset = repeated((X, Y), 20)

In [ ]:
dataz = [(X, Y)]
W = param(rand(1))
b = param(rand(1))
model = Dense(W, b, σ)
loss(x, y) = mse(model(x), y)
# Flux.crossentropy
opt = Descent(0.1)
accuracy(x, y) = mean( round.(model(x)) .== y )

typeof(model)

In [ ]:
params(model)

In [ ]:
loss(X, Y)

In [ ]:
accuracy(X, Y)

In [ ]:
modeldataz(x) = x |> model |> data |> permutedims
# modeldataz(x) = permutedims(data(model(x)))

In [ ]:
modelX = modeldataz(X)
modelapples = modeldataz(g_apples')
modelbanans = modeldataz(g_banans')

plot(modelX, legend = false)
hline!([0.5])
p1 = yaxis!((0,1))

curv = [-1:0.01:1;]' |> modeldataz
plot( [-1:0.01:1;], curv, label="Model", legend = :topleft, lw=3)

scatter!(g_apples, modelapples, label="Apple", colour = :red)
scatter!(g_banans, modelbanans, label="Banana",colour = :yellow)
hline!([0.5], lab = "", legend = :topleft)
p2 = xaxis!("green")

plot(p1, p2)

In [ ]:
savefig("C:/Users/User/Desktop/trained1.png")

In [ ]:
train!(loss, params(model), dataz, opt)

In [ ]:
model.W, model.b

In [ ]:
for i in 1:7000
    train!(loss, params(model), dataz, opt)
end

model.W, model.b

In [ ]:
loss(X, Y)

In [ ]:
accuracy(X, Y)

In [ ]:
model(X)

## Testing

In [ ]:
using Images

function remove_background(img)
    mtrx = copy( channelview(img) )
    for i = 1:size(mtrx, 2), j = 1:size(mtrx, 3)
        if reduce(&, mtrx[:,i,j] .> [0.8, 0.8, 0.8])
            mtrx[:,i,j] .= [0.5, 0.5, 0.5]
        end
    end
    colorview(RGB, mtrx)
end

function classifier(img)
    gbimg = remove_background(img)
    greenmean = [ mean(float.(c.(gbimg))) for c = [red,green,blue] ]
    answ = data( model( [ greenmean[2] ]' ) )[1]
    fr = answ > 0.5 ? "Banana" : "Apple"
    
    "$fr $(round(200abs(0.5-answ)))%"
end

In [ ]:
fnames = [
    "data/10_100.jpg",
    "data/107_100.jpg",
    "data/yellow_apple_2.jpg",
    "data/8_100.jpg",
    "data/104_100.jpg", 
    "data/3_100.jpg"
]
# массив картинок
fruits = [load(fname) for fname in fnames]

hcat(fruits...) 

In [ ]:
classifier.(fruits)

In [ ]:
function classifier2(img)
    gbimg = remove_background(img)
    greenmean = [ mean(float.(c.(gbimg))) for c = [red,green,blue] ]
    answ = data( model( greenmean[2:3] ) )[1]
    fr = answ > 0.5 ? "Banana" : "Apple"
    
    "$fr $(round(200abs(0.5-answ)))%"
end

In [ ]:
classifier2.(fruits)

# Two features

In [ ]:
X = [g_apples b_apples; g_banans b_banans] |> permutedims

dataset2 = [ (X, Y) ]

W = param(rand(1,2))
b = param(rand(1))
model = Dense(W, b, σ)

In [ ]:
model.W, model.b

In [ ]:
for i in 1:2000
    train!(loss, params(model), dataset2, opt)
end

model.W, model.b

In [ ]:
loss(X, Y)

In [ ]:
accuracy(X, Y)

In [ ]:
savefig("C:/Users/User/Desktop/trained3D.png")

In [ ]:
scatter(X[1,:]', X[2,:]', lab = "", color = :blue)

In [ ]:
scatter(X[1,:]', X[2,:]', Y, legend = false, color = :blue)
plot!([0.2, 0.2, 0.5, 0.2, 0.2], [0.2, 0.5, 0.5, 0.5, 0.2], 0.5ones(5) )

# MNIST

In [ ]:
using Images
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
# using CuArrays

# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...);
hcat(imgs[1:10]...)

In [ ]:
hcat(imgs[1:10]...)

In [ ]:
labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) 

In [ ]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

In [ ]:
dataset = repeated((X, Y), 20)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

In [ ]:
for i = 1:10
    Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))
end

In [ ]:
accuracy(X, Y)

In [ ]:
# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...)
tY = onehotbatch(MNIST.labels(:test), 0:9)

accuracy(tX, tY)